# Data preparation and model training for scPheno

- __Data preparation__: scPheno accept three kinds of input files. 
    1. Expression matrix in the MTX format, where rows are cells and columns are genes.
    2. Cell type/state annotation in the tsv/csv format, where a row is the annotation of one cell.
    3. Phenotypic condition annotation in the tsv/csv format, where a row is the annotation of one cell.

    This tutorial uses the COVID-19 dataset provided in the paper [Jeong Seok Lee, et al, 2020](https://doi.org/10.1126/sciimmunol.abd1554). The data in the H5DF format can be downloaded from [here](https://s3.us-west-2.amazonaws.com/atlas.fredhutch.org/data/hutch/covid19/downloads/lee_2020_processed.HDF5). Users can check the R script prepare_data.R In the [Lee_data2](https://github.com/ZengFLab/scPheno/tree/main/Lee_data2) directory for the instructions on preparing the required input files for the Lee's data.

- __Model training__: The following command is used to train a scPheno model for the Lee's data. Users can run  `python scPheno.py -h`  for more details on the scPheno parameters.


In [ ]:
%%bash

python scPheno.py --sup-data-file ~/Project/Lee_data2/lee_dataset.mtx \
                        --sup-label-file ~/Project/Lee_data2/lee_dataset_label_text.txt \
                        --sup-condition-file ~/Project/Lee_data2/lee_dataset_disease_text.txt \
                        -lr 0.00001 \
                        -n 30 \
                        -bs 1000 \
                        --aux-loss \
                        --validation-fold 10 \
                        --cuda \
                        --save-model Lee_model_type.pth




# Compute the cell state-phenotype embedding of cells

In general, scPheno can compute three kinds of low-dimension cell embeddings.
- The joint cell embedding with the consideration of transcriptional variations in both cell type/state and phenotype.
- The cell embedding where only transcriptional variations in cell type/state are of interest.
- The cell embedding where only transcriptional variations in cell phenotype are of interest.


With the trained model and the expression matrix of cells in the MTX format, users can use the function __latent_embedding__ to compute the above three kinds of cell embeddings. The following example demonstrates the steps to compute the cell embeddings for the Lee's data.

First, the required Python packages are imported.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearnex import patch_sklearn
patch_sklearn()

import numpy as np
import pandas as pd
from scipy.io import mmread
from collections import Counter

from scPheno import scPheno
from utils.scdata_cached import setup_data_loader, SingleCellCached, transform_label2class, label2class_encoder, transform_class2label

import torch
import torch.nn.functional as ft
from torch.utils.data import DataLoader

import pyro.distributions as dist

from sklearn.preprocessing import StandardScaler, Normalizer, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, ConfusionMatrixDisplay

import umap

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Second, the trained model and the expression matrix are loaded.

- __batch_size__: In the application phase, batch_size is not needed to be the same as the batch_size in the training phase. Users can use a large batch size in the application phase for high-speed processing.
- __use_float64__: This option is useful when processing an extremely large dataset where the likelihood score can be overflowed if 32-bit float precision is used. This option shall be consistent in the training phase and the application phase. The error will occur if the model is trained with 32-bit float precision but users use it to process data in the 64-bit float precision and vice verse. 
- __use_cuda__: This option shall keep the same as the usage of use_cuda option in the training phase. This option ensures that the model and data are on the same device. Specifically, if the model is trained on a GPU device, the data shall be loaded onto the GPU device. 

In [2]:
ModelPath = 'Lee_model_type.pth'
DataPath='/home/zengbio/Project/Lee_data2/lee_dataset.mtx'
LabelPath=None
ConditionPath=None


In [3]:
# load model
model = torch.load(ModelPath)

batch_size = 100

use_float64 = False
use_cuda = True

In [4]:
model.cond2index.classes_

array(['COVID-19', 'healthy', 'other'], dtype=object)

In [53]:
# load data
data_cached = SingleCellCached(DataPath, LabelPath, ConditionPath, model.class2label, model.index2cond, 'condition', use_cuda=False, use_float64 = use_float64)
data_loader = DataLoader(data_cached, batch_size = batch_size, shuffle = False)

In the default setting, the latent dimension of scPheno is 50. To obtain the 2D visualization of cell embeddings, users should use [UMAP](https://umap-learn.readthedocs.io/en/latest/index.html) on the low-dimension embeddings provided by scPheno. Users can check the R script plot_scp_umap.R in the [Lee_data2](https://github.com/ZengFLab/scPheno/tree/main/Lee_data2) directory for the instructions on how to add the 2D positions to the Seurat object for visualization.

In [54]:
# predict conditions
embeds = []
scores = []

embeds_state = []
embeds_condition = []
# use the appropriate data loader
for xs,ys,ks in data_loader:
    # use classification function to compute all predictions for each batch
    if use_cuda:
        xs = xs.cuda()

    zs = model.latent_embedding(xs)
    _, kscores = model.predicted_condition(xs)

    zs_state = model.latent_embedding(xs, use_state=True, use_condition=False)
    zs_condition = model.latent_embedding(xs, use_state=False, use_condition=True)

    if use_cuda:
        zs = zs.cpu().detach().numpy()
        zs_state = zs_state.cpu().detach().numpy()
        zs_condition = zs_condition.cpu().detach().numpy()
        kscores = kscores.cpu().detach().numpy()
    else:
        zs = zs.detach().numpy()
        zs_state = zs_state.detach().numpy()
        zs_condition = zs_condition.detach().numpy()
        kscores = kscores.detach().numpy()

    embeds.append(zs)
    embeds_state.append(zs_state)
    embeds_condition.append(zs_condition)
    scores.append(kscores)

embeds = np.concatenate(embeds, axis=0)
embeds_state = np.concatenate(embeds_state, axis=0)
embeds_condition = np.concatenate(embeds_condition, axis=0)
scores = np.concatenate(scores, axis=0)

In [ ]:
%%time
pos = umap.UMAP(metric = 'cosine').fit_transform(embeds)
pos_state = umap.UMAP(metric = 'cosine').fit_transform(embeds_state)
pos_condition = umap.UMAP(metric = 'cosine').fit_transform(embeds_condition)

In [57]:
np.savetxt('/home/zengbio/Project/Lee_data2/lee_dataset_umap.txt', pos, delimiter=',')
np.savetxt('/home/zengbio/Project/Lee_data2/lee_dataset_umap_state.txt', pos_state, delimiter=',')
np.savetxt('/home/zengbio/Project/Lee_data2/lee_dataset_umap_condition.txt', pos_condition, delimiter=',')
